# RDD creation

#### [Introduction to Spark with Python, by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

In this notebook we will introduce two different ways of getting data into the basic Spark data structure, the **Resilient Distributed Dataset** or **RDD**. An RDD is a distributed collection of elements. All work in Spark is expressed as either creating new RDDs, transforming existing RDDs, or calling actions on RDDs to compute a result. Spark automatically distributes the data contained in RDDs across your cluster and parallelizes the operations you perform on them.

#### References

The reference book for these and other Spark related topics is *Learning Spark* by Holden Karau, Andy Konwinski, Patrick Wendell, and Matei Zaharia.  

The KDD Cup 1999 competition dataset is described in detail [here](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99).

## SparkContext

In [1]:
'''
Punto de acceso. Para realizar operaciones necesitamos un Context: SparkContext, SQLContext...
Dependerá del tipo de operación
al principio estaba SparkContext y se usaba para operaciones con RDDs, despues salio SparkSession,
para RDDs, Dataframes y Datsets. SparkSession contempla internamente el SparkContext, HiveContext, SQLContext...
Para solucionar esto, tenemos el SparkSession. Que nos sirve para todos los contextos
Eso sería lo más correcto, ya que establecemos una sesión con el nodo maestro.
'''
import pyspark
from pyspark import SparkContext
sc =SparkContext()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
sc=spark.sparkContext

In [3]:
'''
Disponemos de una interfaz gráfica en el sparkcontext donde se encuentran todos los jobs ejecutados.
'''
sc

<SparkContext master=yarn appName=pyspark-shell>

## Getting the data files  

In this notebook we will use the reduced dataset (10 percent) provided for the KDD Cup 1999, containing nearly half million network interactions. The file is provided as a *Gzip* file that we will download locally.  

In [4]:
'''
Descripción del dataset
This is the data set used for The Third International Knowledge Discovery and Data Mining Tools Competition, which was held
in conjunction with KDD-99 The Fifth International Conference on Knowledge Discovery and Data Mining. The competition task was
to build a network intrusion detector, a predictive model capable of distinguishing between ``bad'' connections, called intrusions
or attacks, and ``good'' normal connections. This database contains a standard set of data to be audited, which includes a wide
variety of intrusions simulated in a military network environment.

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
'''

import urllib
f = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz",
                               "/home/daniel/kddcup.data_10_percent.gz")

## Creating a RDD from a file  

The most common way of creating an RDD is to load it from a file. Notice that Spark's `textFile` can handle compressed files directly.    

In [5]:
'''
Probar a leerlo desde la ruta normal (/home/daniel/kddcup.data_10_percent.gz)
Llegar hasta el count, para que vean lo que es el lazy evaluation
Tenemos que pasar a HDFS!!
hadoop fs -mkdir /user/input
hadoop fs -put -f kddcup.data_10_percent.gz /user/input
'''
#data_file = "/home/daniel/kddcup.data_10_percent.gz/kddcup.data_10_percent.gz"
data_file = "/user/input/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [6]:
type(raw_data)

pyspark.rdd.RDD

Now we have our data file loaded into the `raw_data` RDD.

Without getting into Spark *transformations* and *actions*, the most basic thing we can do to check that we got our RDD contents right is to `count()` the number of lines loaded from the file into the RDD.  

In [7]:
raw_data.count()

494021

We can also check the first few entries in our data.  

In [23]:
'''
Cada linea es una cadena de texto que tendremos que parsear mas adelante
'''
raw_data.take(5)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In the following notebooks, we will use this raw data to learn about the different Spark transformations and actions.  

## Creating and RDD using `parallelize`

Another way of creating an RDD is to parallelize an already existing list.  

In [34]:
a = range(100)

data = sc.parallelize(a)
type(data)

pyspark.rdd.PipelinedRDD

As we did before, we can `count()` the number of elements in the RDD.

In [35]:
data.count()

100

As before, we can access the first few elements on our RDD.  

In [36]:
data.take(5)

[0, 1, 2, 3, 4]

## Get data and partitions

In [37]:
'''
Por defecto hace 10 particiones, tb se lo podemos especificar
data = sc.parallelize(a, 10)
'''
rddCollect = data.collect()

print("Number of Partitions: "+str(data.getNumPartitions()))
print("Action: First element: "+str(data.first()))
print(rddCollect)

Number of Partitions: 2
Action: First element: 0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
